In [ ]:
import numpy as np

In [ ]:
# Day 6
with open("input6", "r") as fp:
    lines = fp.readlines()
all_groups = []
current = []
for line in lines:
    if line.strip() == "":
        all_groups.append(current)
        current = []
    else:
        current.append(line.strip())
all_groups.append(current)
all_strings = ["".join(a) for a in all_groups]
all_unique = [len(np.unique(list(a))) for a in all_strings]
print(np.sum(all_unique))
all_every = []
for g in all_groups:
    if len(g) == 0:
        continue
    a = set(list(g[0]))
    if len(g) > 1:
        for b in g[1:]:
            a = a.intersection(set(list(b)))
    all_every.append(len(a))
print(np.sum(all_every))

In [ ]:
# Day 5
with open("input5", "r") as fp:
    lines = fp.readlines()
seat_nr = []
for line in lines:
    row = line[:7].replace("B","1").replace("F", "0")
    col = line[7:10].replace("R","1").replace("L", "0")
    row = int(row, base=2)
    col = int(col, base=2)
    id = row*8+col
    seat_nr.append([row, col, id])
print(np.max(np.array(seat_nr)[:,2]))
ids = [s[2] for s in seat_nr]
ids.sort()
np.argmax(np.diff(ids))
print(ids[463:465])